# From ATOM feed to pandas

In [11]:
import pandas as pd
import feedparser

In [12]:
# Earthquakes for one month period to today
#
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.atom"
feed = feedparser.parse(url)

In [13]:
from typing import Any

entries: list[dict[str, Any]] = [dict(e) for e in feed.entries]
entries[0]

{'id': 'urn:earthquake-usgs-gov:ci:41131151',
 'guidislink': True,
 'link': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41131151',
 'title': 'M 2.2 - 21 km N of Barstow, CA',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.atom',
  'value': 'M 2.2 - 21 km N of Barstow, CA'},
 'updated': '2025-12-02T21:47:55.654Z',
 'updated_parsed': time.struct_time(tm_year=2025, tm_mon=12, tm_mday=2, tm_hour=21, tm_min=47, tm_sec=55, tm_wday=1, tm_yday=336, tm_isdst=0),
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41131151'}],
 'summary': '<dl><dt>Time</dt><dd>2025-12-02 21:44:02 UTC</dd><dd>2025-12-02 21:44:02 UTC at epicenter</dd><dt>Location</dt><dd>35.090&deg;N 117.034&deg;W</dd><dt>Depth</dt><dd>0.69 km (0.43 mi)</dd></dl>',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://earthquake.usgs.gov/e

In [14]:
df_quakes = pd.json_normalize(entries)
df_quakes.head(2)

,id,guidislink,link,title,updated,updated_parsed,links,summary,georss_elev,tags,title_detail.type,title_detail.language,title_detail.base,title_detail.value,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value,where.type,where.coordinates
0,urn:earthquake-usgs-gov:ci:41131151,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 2.2 - 21 km N of Barstow, CA",2025-12-02T21:47:55.654Z,"(2025, 12, 2, 21, 47, 55, 1, 336, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-02 21:44:02 UTC</...,-690,"[{'term': 'Past Hour', 'scheme': None, 'label'...",text/plain,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 2.2 - 21 km N of Barstow, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-02 21:44:02 UTC</...,Point,"(-117.03383333333, 35.090166666667)"
1,urn:earthquake-usgs-gov:nc:75273281,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 0.8 - 5 km NW of The Geysers, CA",2025-12-02T21:36:18.602Z,"(2025, 12, 2, 21, 36, 18, 1, 336, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-02 21:34:43 UTC</...,-3319.9999332428,"[{'term': 'Past Hour', 'scheme': None, 'label'...",text/plain,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 0.8 - 5 km NW of The Geysers, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-02 21:34:43 UTC</...,Point,"(-122.80200195312, 38.805000305176)"


1. How many quakes were there?

In [9]:
# Quake count


In [10]:
# Add a mag column containing the magnitude of the quake extracted from title using regex
#
mag = (
    df_quakes["title"]
    .str.extract(r"\bM\s*([0-9]+(?:\.[0-9]+)?)")[0]
    .astype(float)
)

# mag column now holds, for example, 2.3 for "M 2.3 - 11 km SSE of Maria Antonia, Puerto Rico."
#
df_quakes["mag"] = mag
df_quakes.head(1)

,id,guidislink,link,title,updated,updated_parsed,links,summary,georss_elev,tags,...,title_detail.language,title_detail.base,title_detail.value,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value,where.type,where.coordinates,mag
0,urn:earthquake-usgs-gov:ci:41131151,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 2.2 - 21 km N of Barstow, CA",2025-12-02T21:47:55.654Z,"(2025, 12, 2, 21, 47, 55, 1, 336, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-02 21:44:02 UTC</...,-690,"[{'term': 'Past Hour', 'scheme': None, 'label'...",...,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 2.2 - 21 km N of Barstow, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-02 21:44:02 UTC</...,Point,"(-117.03383333333, 35.090166666667)",2.2


2. Filter df_quakes for mag 4.0 or larger and write the Title, Magnitude, (Longitude, Latitude) to a csv named data\quakes_ge_4.csv